## Instalacion Librerias

In [ ]:
pip install requests

In [ ]:
pip install beautifulsoup4

In [ ]:
pip install pandas 

In [ ]:
pip install logging

## Importar Librerias

In [20]:
import requests
import logging
from bs4 import BeautifulSoup
import pandas as pd

## Objetivo

El objetivo del proyecto consiste en verificar la información contenida en esta página web, identificando qué datos se pueden extraer. Además, buscamos determinar la frecuencia con la que esta información se actualiza, identificar los cambios específicos que se producen y mantenernos informados sobre las últimas noticias relacionadas.

<img src="images/01.png" alt="Descripción de la foto" width="800" height="600">


Este script realiza una solicitud GET a la página web especificada para obtener su contenido HTML. Utilizando la biblioteca BeautifulSoup para parsear dicho contenido, el objetivo es identificar la estructura del HTML de la página. Este proceso permite una inspección inicial del código fuente, facilitando la identificación de los elementos específicos de la página que podrían ser de interés para extracciones de datos posteriores. El script verifica primero la respuesta del servidor para asegurar que la solicitud fue exitosa antes de proceder con el análisis del HTML.

In [24]:
# Configurar el nivel de log
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

# URL de la página web
url = "https://www.constructorabolivar.com/proyectos-vivienda"

try:
    # Realizar la solicitud GET
    respuesta = requests.get(url, timeout=5)  # Esperar 5 segundos como máximo

    # Verificar el estado de la solicitud
    if respuesta.status_code == 200:
        # Parsear el contenido HTML
        soup = BeautifulSoup(respuesta.text, 'html.parser')
        
        # Mostrar una muestra del HTML
        html_sample = soup.prettify()[:1000]  # Limitado a los primeros 1000 caracteres
        logging.info("Muestra del HTML:\n" + html_sample)
        
    else:
        logging.error(f"Error en la solicitud: Estado {respuesta.status_code}")
except requests.RequestException as e:
    logging.error(f"Error al realizar la solicitud: {e}")


INFO: Muestra del HTML:
<!DOCTYPE html>
<html dir="ltr" lang="es" prefix="content: http://purl.org/rss/1.0/modules/content/  dc: http://purl.org/dc/terms/  foaf: http://xmlns.com/foaf/0.1/  og: http://ogp.me/ns#  rdfs: http://www.w3.org/2000/01/rdf-schema#  schema: http://schema.org/  sioc: http://rdfs.org/sioc/ns#  sioct: http://rdfs.org/sioc/types#  skos: http://www.w3.org/2004/02/skos/core#  xsd: http://www.w3.org/2001/XMLSchema# ">
 <head>
  <!-- Google Tag Manager -->
  <script>
   (function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
  new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
  j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
  'https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);
  })(window,document,'script','dataLayer','GTM-PNJFMZD');
  </script>
  <!-- Google Tag Manager -->
  <script>
   (function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
  new Date().getTime(),event:'gtm.js'});var

<div align="center">
    <img src="images/02.png" alt="Descripción de la foto" width="800" height="600">
</div>

Buscamos dentro de la página elementos "strong" que estén anidados directamente bajo elementos "h3", los cuales a su vez se encuentran dentro de "div" con la clase card-item-title. Al iterar sobre estos elementos y imprimir el texto que contienen, el script busca evidencia de contenido dinámico. La presencia de placeholders como ${item.title} en la respuesta impresa sugiere que la página podría estar cargando contenido dinámicamente mediante JavaScript, ya que estos placeholders suelen reemplazarse con datos reales en tiempo de ejecución por scripts del lado del cliente.

In [ ]:
# URL de la página web
url = "https://www.constructorabolivar.com/proyectos-vivienda"

# Realizar la solicitud GET
respuesta = requests.get(url)

# Verificar el estado de la solicitud
if respuesta.status_code == 200:
    # Parsear el contenido HTML
    soup = BeautifulSoup(respuesta.text, 'html.parser')
    # Buscar todos los elementos 'strong' directamente dentro de 'h3' dentro de los contenedores 'div' con clase 'card-item-title'
    titulos = soup.select('div.card-item-title h6 strong')
    # Iterar sobre cada título y imprimir el texto
    for titulo in titulos:
        print(titulo.text.strip())
else:
    print("Error en la solicitud: Estado", respuesta.status_code)

De igual manera, podemos asignarle un nombre al campo que recupera, sin embargo, de esta manera no podriamos obtener el nombre real.

In [ ]:

# URL de la página web
url = "https://www.constructorabolivar.com/proyectos-vivienda"

# Realizar la solicitud GET
respuesta = requests.get(url)

# Verificar el estado de la solicitud
if respuesta.status_code == 200:
    # Parsear el contenido HTML
    soup = BeautifulSoup(respuesta.text, 'html.parser')
    # Buscar todos los elementos 'strong' directamente dentro de 'h3' dentro de los contenedores 'div' con clase 'card-item-title'
    titulos = soup.select('div.card-item-title h3 strong')
    # Iterar sobre cada título y imprimir el texto
    for titulo in titulos:
        # Aquí se imprime la cadena literal "${item.title}"
        print(titulo.text.strip())
        # Aquí se reemplaza la cadena literal "${item.title}" por "Proyecto XYZ"
        texto_reemplazado = titulo.text.strip().replace("${item.title}", "Proyecto XYZ")
        print(texto_reemplazado)
else:
    print("Error en la solicitud: Estado", respuesta.status_code)

Al enfrentarnos a la limitación de no poder recuperar información de manera eficaz mediante métodos tradicionales de scraping HTML, debido a la naturaleza dinámica de la página objetivo, optamos por acceder directamente a una API proporcionada por el sitio web. Este enfoque nos permite obtener la información de manera estructurada y completa durante el proceso de carga de la página, aprovechando el formato JSON para facilitar el análisis y extracción de datos relevantes.

El código realiza una solicitud GET a la URL específica de la API, que devuelve los datos de los proyectos en formato JSON. Tras verificar una respuesta exitosa del servidor (código de estado 200), el script parsea esta respuesta JSON para iterar sobre cada objeto de proyecto individualmente. Durante esta iteración, extrae y muestra datos clave de cada proyecto, incluyendo identificadores únicos, títulos, ciudades, precios, estados, y más, así como URLs para logos e imágenes destacadas. Además, proporciona información sobre el tipo de vivienda y la disponibilidad de subsidios, culminando con la URL para una vista detallada de cada proyecto.

Este método nos permite obtener una visión detallada y actualizada de los proyectos disponibles, superando los desafíos que presenta el scraping de sitios web dinámicos y aprovechando la eficiencia y estructura que ofrecen las APIs para la recuperación de datos.

La informacion de la API se obtiene de la siguiente manera, si no aparece, recargar la pagina

<div align="center">
    <img src="images/03.png" alt="Obtencion Extension xhr" width="800" height="600">
</div>

Informacion del JSON correspondiente al xhr de la API

<div align="center">
    <img src="images/04.png" alt="Json Response" width="800" height="600">
</div>

In [ ]:
# URL de la API
url = "https://www.constructorabolivar.com/api/proyectos2/all/all/all?_format=json"

# Realizar la solicitud GET
respuesta = requests.get(url)

# Verificar si la solicitud fue exitosa
if respuesta.status_code == 200:
    # Parsear la respuesta JSON
    proyectos = respuesta.json()
    # Iterar sobre cada proyecto e imprimir algunos campos
    for proyecto in proyectos:
        print(f"ID: {proyecto['nid']}")
        print(f"Título: {proyecto['title']}")
        print(f"Departamento: {proyecto['field_departamento']}")
        print(f"Ciudad: {proyecto['field_ciudad']}")
        print(f"Precio: {proyecto['field_precio']}")
        print(f"Precio SMMLV: {proyecto['field_precio_smmlv']}")
        print(f"Sector: {proyecto['field_sector']}")
        print(f"Barrio: {proyecto['field_barrio']}")
        print(f"Estado: {proyecto['field_estados']}")
        print(f"Area Minima: {proyecto['field_area_minima']}")
        print(f"Area Privada Minima: {proyecto['field_area_privada_minima']}")
        print(f"Alcobas: {proyecto['field_alcobas']}")
        print(f"Logo URL: {proyecto['field_logo']}")
        print(f"Imagen Destacada URL: {proyecto['field_imagen_destacada']}")
        print(f"Tipo de Vivienda: {proyecto['field_tipo_de_vivienda']}")
        print(f"URL de Vista: {proyecto['view_node']}")
        print(f"Descripciones Destacadas: {proyecto.get('field_descripcion_destacada_1', '')}, {proyecto.get('field_descripcion_destacada_2', '')}, {proyecto.get('field_descripcion_destacada_3', '')}, {proyecto.get('field_descripcion_destacada_4', '')}")
        print("------------------------------------------------")
else:
    print("Error en la solicitud: Estado", respuesta.status_code)


Para obtener una visión completa de los proyectos, es esencial acceder a los detalles específicos contenidos dentro de cada página de proyecto. Aunque ya hemos recopilado la información principal de los proyectos, nos falta explorar la información detallada que se encuentra al interior de cada uno de ellos.

Por esta razón, hemos ajustado nuestro código para que ahora incluya la capacidad de extraer la URL de cada proyecto. Este paso nos permite adentrarnos en cada página individual de proyecto, donde podremos recabar datos más específicos y enriquecer nuestro conjunto de información con detalles más profundos sobre cada proyecto.

<div align="center">
    <img src="images/05.png" alt="Obtencion Extension xhr" width="800" height="600">
</div>

In [ ]:
# URL base del sitio web
url_base = "https://www.constructorabolivar.com"

# URL de la API
url_api = "https://www.constructorabolivar.com/api/proyectos2/all/all/all?_format=json"

# Realizar la solicitud GET a la API
respuesta = requests.get(url_api)

# Función para obtener la información detallada de un proyecto
def obtener_info_proyecto(url_proyecto):
    respuesta = requests.get(url_proyecto)
    if respuesta.status_code == 200:
        soup = BeautifulSoup(respuesta.text, 'html.parser')
        # Ajusta el selector para obtener la descripción
        elemento_descripcion = soup.find('div', class_='descripcion-clase')  # Cambiar por el selector correcto
        if elemento_descripcion:  # Verificar si se encontró el elemento
            descripcion = elemento_descripcion.text.strip()
        else:
            descripcion = "Descripción no encontrada"
        return descripcion
    else:
        return "No se pudo obtener la información"

if respuesta.status_code == 200:
    proyectos = respuesta.json()
    for proyecto in proyectos:
        # URL completa del proyecto
        url_proyecto_completa = url_base + proyecto['view_node']
        print(f"URL del Proyecto: {url_proyecto_completa}")

        # Obtener información detallada de cada proyecto
        info_adicional = obtener_info_proyecto(url_proyecto_completa)
        print(f"Información adicional: {info_adicional}")
        print("------------------------------------------------")
else:
    print("Error en la solicitud: Estado", respuesta.status_code)

Hemos integrado ambos procedimientos con el objetivo de capturar no solo los datos generales de los proyectos, sino también los detalles específicos accesibles a través de sus respectivas páginas web. Este enfoque nos permite extraer la URL

In [ ]:
# URL base del sitio web para completar las URLs relativas
url_base = "https://www.constructorabolivar.com"

# Función para obtener información detallada de cada proyecto
def obtener_info_proyecto(url_proyecto):
    respuesta = requests.get(url_proyecto)
    if respuesta.status_code == 200:
        soup = BeautifulSoup(respuesta.text, 'html.parser')
        # Aquí puedes ajustar los selectores según la estructura específica de la página de cada proyecto
        # y la información adicional que deseas obtener. Este es un ejemplo para obtener una descripción:
        elemento_descripcion = soup.find('div', class_='alguna-clase-para-la-descripcion')  # Ajusta este selector
        descripcion = elemento_descripcion.text.strip() if elemento_descripcion else "Descripción no encontrada"
        return descripcion
    else:
        return "No se pudo obtener la información del proyecto"

# URL de la API
url_api = "https://www.constructorabolivar.com/api/proyectos2/all/all/all?_format=json"

# Realizar la solicitud GET a la API
respuesta = requests.get(url_api)

if respuesta.status_code == 200:
    proyectos = respuesta.json()
    for proyecto in proyectos:
        # Imprimir información básica obtenida de la API
        print(f"ID: {proyecto['nid']}")
        print(f"Título: {proyecto['title']}")

        # Completar la URL relativa del proyecto
        url_proyecto = url_base + proyecto['view_node']
        print(f"URL del Proyecto: {url_proyecto}")
        info_adicional = obtener_info_proyecto(url_proyecto)
        print(f"Información adicional: {info_adicional}")
        print("------------------------------------------------")
else:
    print("Error en la solicitud: Estado", respuesta.status_code)

En este código, se realiza una integración avanzada de solicitudes HTTP y análisis HTML para extraer y compilar una amplia gama de información sobre proyectos de construcción disponibles a través de la API pública y las páginas web individuales de cada proyecto. La estrategia implementada consiste en dos fases principales:

**Recopilación de Datos Generales de la API:** Inicialmente, se efectúa una solicitud GET a la API proporcionada por el sitio web constructorabolivar.com, donde se recoge información general de cada proyecto. Esta información incluye, pero no se limita a, identificadores únicos, títulos, ubicaciones, precios, y enlaces directos a las páginas específicas de cada proyecto.

**Extracción de Detalles Específicos de las Páginas de Proyectos:** Utilizando las URLs específicas de cada proyecto, el código procede a realizar solicitudes individuales para acceder a las páginas correspondientes. A través de técnicas de scraping con BeautifulSoup, se extraen detalles más profundos que no están presentes en la respuesta de la API, como descripciones detalladas, que ofrecen un mayor contexto y valor agregado al conjunto de datos.

Todos los datos recopilados se organizan y almacenan en un DataFrame de pandas, lo que facilita un posterior análisis y manipulación de los datos. Este DataFrame contiene tanto los datos generales obtenidos directamente de la API como los detalles específicos extraídos de las páginas web de cada proyecto.

In [ ]:
# URL base del sitio web para completar las URLs relativas
url_base = "https://www.constructorabolivar.com"

# Función para obtener información detallada de cada proyecto
def obtener_info_proyecto(url_proyecto):
    try:
        respuesta = requests.get(url_proyecto)
        if respuesta.status_code == 200:
            soup = BeautifulSoup(respuesta.text, 'html.parser')
            # Ajusta el selector para obtener la descripción
            elemento_descripcion = soup.find('div', class_='alguna-clase-para-la-descripcion')
            descripcion = elemento_descripcion.text.strip() if elemento_descripcion else "Descripción no encontrada"
            return descripcion
        else:
            return "No se pudo obtener la información del proyecto"
    except Exception as e:
        return f"Error al obtener la información: {e}"

# URL de la API
url_api = "https://www.constructorabolivar.com/api/proyectos2/all/all/all?_format=json"

# Realizar la solicitud GET a la API
respuesta = requests.get(url_api)

# Lista para almacenar la información de cada proyecto
proyectos_info = []

if respuesta.status_code == 200:
    proyectos = respuesta.json()
    for proyecto in proyectos:
        # Formar la URL completa de la vista del proyecto y obtener información adicional
        url_proyecto = url_base + proyecto['view_node']
        info_adicional = obtener_info_proyecto(url_proyecto)
        
        # Almacenar información del proyecto en la lista
        proyectos_info.append({
            "ID": proyecto['nid'],
            "Título": proyecto['title'],
            "Departamento": proyecto['field_departamento'],
            "Ciudad": proyecto['field_ciudad'],
            "Precio": proyecto['field_precio'],
            "Precio SMMLV": proyecto['field_precio_smmlv'],
            "Sector": proyecto['field_sector'],
            "Barrio": proyecto['field_barrio'],
            "Estado": proyecto['field_estados'],
            "Area Minima": proyecto['field_area_minima'],
            "Area Privada Minima": proyecto['field_area_privada_minima'],
            "Alcobas": proyecto['field_alcobas'],
            "Logo URL": proyecto['field_logo'],
            "Imagen Destacada URL": proyecto['field_imagen_destacada'],
            "Tipo de Vivienda": proyecto['field_tipo_de_vivienda'],
            "URL de Vista": url_proyecto,
            "Información adicional": info_adicional
        })
else:
    print("Error en la solicitud: Estado", respuesta.status_code)

# Convertir la lista de proyectos en un DataFrame
df_proyectos = pd.DataFrame(proyectos_info)

# Muestra las primeras filas del DataFrame para verificar
print(df_proyectos.head())

In [33]:
# Mostrar las primeras 5 filas del DataFrame
df_proyectos.head(5)

,ID,Título,Departamento,Ciudad,Precio,Precio SMMLV,Sector,Barrio,Estado,Area Minima,Area Privada Minima,Alcobas,Logo URL,Imagen Destacada URL,Tipo de Vivienda,URL de Vista,Información adicional
0,1727,Loto - Vivero Parque Residencial,Valle del Cauca,Cali y sus alrededores,577857946,,Cerca a Valle del Lili,Valle del Lili,Lanzamiento,"101,73","93,42",2,https://cbolivarstoragedev.blob.core.windows.n...,https://cbolivarstoragedev.blob.core.windows.n...,Apartamentos,https://www.constructorabolivar.com/proyectos-...,No se pudo obtener la información del proyecto
1,1760,Almendros del Edén,Cundinamarca,Bogotá y sus alrededores,260000000,,Funza,Funza,Lanzamiento,43,43,2,https://cbolivarstoragedev.blob.core.windows.n...,https://cbolivarstoragedev.blob.core.windows.n...,Apartamentos,https://www.constructorabolivar.com/proyectos-...,Descripción no encontrada
2,1791,Puerto Celesta - Ciudad del Puerto,Atlántico,Soledad - Atlántico,126400000,90 SMMLV,Calle 30,Ciudad del Puerto,Lanzamiento,41,37,2,https://cbolivarstoragedev.blob.core.windows.n...,https://cbolivarstoragedev.blob.core.windows.n...,Apartamentos,https://www.constructorabolivar.com/proyectos-...,Descripción no encontrada
3,1709,Arroyuelo - Entrerios Ciudad Campestre,Valle del Cauca,Cali y sus alrededores,126360000,90 SMMLV,El Cerrito,,Lanzamiento,30,27.45,,https://cbolivarstoragedev.blob.core.windows.n...,https://cbolivarstoragedev.blob.core.windows.n...,Casas,https://www.constructorabolivar.com/proyectos-...,Descripción no encontrada
4,1766,Luciérnagas - Ciudad del Valle,Valle del Cauca,Cali y sus alrededores,223236000,150 SMMLV,Ciudad del Valle,Ciudad del Valle,Lanzamiento,45.58,40.85,1,https://cbolivarstoragedev.blob.core.windows.n...,https://cbolivarstoragedev.blob.core.windows.n...,Casas,https://www.constructorabolivar.com/proyectos-...,Descripción no encontrada


## Objetivo 2

Realizar el mismo proceso del objetivo anterior, pero esta ves contemplando otra constructora

In [ ]:

# URL de la página web
url = "https://marval.com.co/proyectos/"

# Realizar la solicitud GET con la verificación de SSL desactivada
respuesta = requests.get(url, verify=False)

# Verificar el estado de la solicitud
if respuesta.status_code == 200:
    # Parsear el contenido HTML
    soup = BeautifulSoup(respuesta.text, 'html.parser')
    # Buscar todos los elementos 'a' dentro de 'h2' con clase 'item-title', que están dentro de los contenedores 'div' con clase 'item-body' y 'flex-grow-1'
    titulos = soup.select('div.item-body.flex-grow-1 h2.item-title a')
    # Iterar sobre cada título y imprimir el texto
    for titulo in titulos:
        print(titulo.text.strip())
else:
    print("Error en la solicitud: Estado", respuesta.status_code)

In [ ]:
# URL de la solicitud AJAX
url = "https://marval.com.co/wp-admin/admin-ajax.php"

# Parámetros de la solicitud identificados
data = {
    'action': 'houzez_loadmore_properties',
    'prop_limit': 9,  # Límite de propiedades por carga
    'paged': 2,  # Número de página o conjunto de resultados a cargar
    'card_version': 'item-v2',  # Versión de la tarjeta de propiedad
    'type': 'proyectos',  # Tipo de propiedades a cargar
    'status': 'en-construccion,en-entregas,lanzamiento,sobre-planos,ultimas-unidades',  # Estado de las propiedades
    'state': '',  # Estado/Departamento (vacío significa sin filtro por estado)
    'city': '',  # Ciudad (vacío significa sin filtro por ciudad)
    'country': '',  # País (vacío significa sin filtro por país)
    'area': '',  # Área (vacío significa sin filtro por área)
    'label': '',  # Etiqueta (vacío significa sin filtro por etiqueta)
    'user_role': '',  # Rol del usuario (vacío significa sin filtro por rol)
    'featured_prop': '',  # Propiedades destacadas (vacío significa sin filtro por destacados)
    'sort_by': 'd_date',  # Ordenar por fecha descendente
    'offset': '',  # Desplazamiento (vacío significa sin desplazamiento específico)
}

# Realizar la solicitud POST
respuesta = requests.post(url, data=data , verify=False)

if respuesta.status_code == 200:
    # Procesar la respuesta
    soup = BeautifulSoup(respuesta.content, 'html.parser')
    # Aquí iría tu código para extraer y procesar la información deseada del contenido HTML
    
    # Ejemplo: imprimir los títulos de los proyectos cargados
    proyectos = soup.find_all('h2', class_='item-title')
    for proyecto in proyectos:
        titulo = proyecto.find('a').text.strip()
        print(titulo)
else:
    print(f"Error al realizar la solicitud: Estado {respuesta.status_code}")

In [ ]:
def cargar_proyectos(pagina):
    url = "https://marval.com.co/wp-admin/admin-ajax.php"
    data = {
        'action': 'houzez_loadmore_properties',
        'prop_limit': 9,
        'paged': pagina,
        'card_version': 'item-v2',
        'type': 'proyectos',
        'status': 'en-construccion,en-entregas,lanzamiento,sobre-planos,ultimas-unidades',
        'sort_by': 'd_date',
    }
    respuesta = requests.post(url, data=data, verify=False)
    if respuesta.status_code == 200:
        return respuesta.content
    else:
        return None

# Iniciar en la página 1
pagina = 1

while True:
    respuesta_html = cargar_proyectos(pagina)
    if respuesta_html:
        soup = BeautifulSoup(respuesta_html, 'html.parser')
        proyectos = soup.find_all('div', class_='item-body flex-grow-1')
        if not proyectos:
            print("No se encontraron más proyectos.")
            break

        for proyecto in proyectos:
            # Extraer el título/nombre del proyecto
            titulo = proyecto.find('h2', class_='item-title').get_text(strip=True)
            
            # Extraer el estado y la etiqueta
            estado_etiquetas = proyecto.find_all('a', class_='label-status') + proyecto.find_all('a', class_='hz-label')
            estado = [etiqueta.get_text(strip=True) for etiqueta in estado_etiquetas]

            # Extraer el precio
            precio = proyecto.find('li', class_='item-price').get_text(strip=True) if proyecto.find('li', class_='item-price') else 'No disponible'

            # Extraer los metros cuadrados
            metros_cuadrados = proyecto.find('li', class_='h-area').get_text(strip=True) if proyecto.find('li', class_='h-area') else 'No disponible'

            print(f"Nombre: {titulo}")
            print(f"Estado/Etiqueta: {', '.join(estado)}")
            print(f"Precio: {precio}")
            print(f"Metros Cuadrados: {metros_cuadrados}")
            print("----------")

        # Cargar la siguiente página
        pagina += 1
    else:
        print(f"Error al cargar la página {pagina}")
        break